In [43]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [44]:
conn = sqlite3.connect('livs.db')  # create db and establish connection

In [45]:
conn.row_factory = sqlite3.Row

In [46]:
curs = conn.cursor()

In [47]:
result = []

In [48]:
for row in curs.execute('select "Kolhydrater_g", "Fett_g", "Protein_g" from livs'):
        result.append(row)

In [49]:
conn.close()

In [50]:
dataSet = np.array(result)
debug = False

In [51]:
if debug:
    print dataSet

In [52]:
def cluster(dataSet):

    kmeans = KMeans(n_clusters=numClusters)
    kmeans.fit(dataSet)

    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_

    if debug:
        print (centroids)
        print (labels)

    return labels

In [55]:
numClusters = 3
labels = cluster(dataSet)

In [56]:
# transpose to work with Plotly-format (don't remember if it is sklear.cluster or plotly that is counterintuitive)
dataToChart = dataSet.transpose()

In [57]:
if debug:
    print dataToChart

In [58]:
# create multi dimensional array of data by label
segmentedData = [[[] for _ in xrange(3)] for _ in xrange(numClusters)]

In [59]:
for num, label in enumerate(labels):
        if debug:
            print (str(num) + ' ' + str(label))
        segmentedData[label][0].append(dataToChart[0][num])
        segmentedData[label][1].append(dataToChart[1][num])
        segmentedData[label][2].append(dataToChart[2][num])

In [60]:
if debug:
        print(segmentedData)

In [61]:
# create traces for plotly
traces = []
baseColor = 100
i = 0
while i < numClusters:
    trace = go.Scatter3d(
        x=segmentedData[i][0],
        y=segmentedData[i][1],
        z=segmentedData[i][2],
        mode='markers',
        marker=dict(
            size=12,
            line=dict(
                color='rgba(baseColor+(i*2), baseColor+(i*2), baseColor+(i*2), 0.14)',
                width=0.5
            ),
            opacity=0.8
        ),
        # @todo: fix names list for plotly
        #text=names
    )
    traces.append(trace)
    i+=1


In [62]:
layout = go.Layout(
        scene=go.Scene(
            xaxis=go.XAxis(title='Carbs', tickprefix='Carbs ', showtickprefix='first'),
            yaxis=go.YAxis(title='Fat', tickprefix='Fat ', showtickprefix='first'),
            zaxis=go.ZAxis(title='Protein', tickprefix='Protein ', showtickprefix='first')
        ),
        height = 800,
        width = 800,
    )

In [63]:
fig = go.Figure(data=traces, layout=layout)    

In [64]:
py.iplot(fig, filename='table1')